# GitVizz GraphSearchTool v2 Demo

**Clean, subgraph-centric API where every method returns a visualizable GraphGenerator**

In [1]:
# Setup
import sys
from pathlib import Path
sys.path.append(str(Path().parent))

from gitvizz import GraphGenerator, GraphSearchTool

## 🏗️ Create Sample Codebase

In [2]:
# FastAPI application codebase
sample_files = [
    {
        "path": "api/server.py",
        "content": '''
from fastapi import FastAPI, HTTPException
from .auth import AuthManager
from .database import UserRepo, PostRepo
from .models import User, Post, CreateUserRequest

class APIServer:
    def __init__(self):
        self.app = FastAPI(title="Blog API")
        self.auth = AuthManager()
        self.user_repo = UserRepo()
        self.post_repo = PostRepo()
    
    def setup_routes(self):
        @self.app.get("/users")
        async def list_users():
            return await self.user_repo.get_all()
        
        @self.app.post("/users")
        async def create_user(request: CreateUserRequest):
            if await self.user_repo.get_by_username(request.username):
                raise HTTPException(400, "User exists")
            return await self.user_repo.create(request)
        
        @self.app.get("/posts/{user_id}")
        async def get_user_posts(user_id: int):
            return await self.post_repo.get_by_user(user_id)
    
    def run(self):
        self.setup_routes()
        return self.app
''',
        "full_path": "/project/api/server.py"
    },
    {
        "path": "api/auth.py",
        "content": '''
import jwt
from datetime import datetime, timedelta
from .models import User
from .database import UserRepo

class AuthManager:
    def __init__(self, secret="jwt-secret"):
        self.secret = secret
        self.algorithm = "HS256"
        self.user_repo = UserRepo()
    
    def create_token(self, user: User) -> str:
        payload = {
            "user_id": user.id,
            "username": user.username,
            "exp": datetime.utcnow() + timedelta(days=1)
        }
        return jwt.encode(payload, self.secret, algorithm=self.algorithm)
    
    def verify_token(self, token: str) -> dict:
        return jwt.decode(token, self.secret, algorithms=[self.algorithm])
    
    async def login(self, username: str, password: str) -> User:
        user = await self.user_repo.get_by_username(username)
        if user and self.check_password(password, user.password_hash):
            return user
        return None
    
    def check_password(self, password: str, hash: str) -> bool:
        return True  # Mock implementation

class TokenValidator:
    def __init__(self, auth_manager: AuthManager):
        self.auth = auth_manager
    
    def validate(self, token: str) -> bool:
        try:
            self.auth.verify_token(token)
            return True
        except:
            return False
''',
        "full_path": "/project/api/auth.py"
    },
    {
        "path": "api/database.py",
        "content": '''
import asyncpg
from typing import List, Optional
from .models import User, Post, CreateUserRequest

class DatabasePool:
    def __init__(self, url: str = "postgresql://localhost/blog"):
        self.url = url
        self.pool = None
    
    async def connect(self):
        self.pool = await asyncpg.create_pool(self.url)
    
    async def get_connection(self):
        return self.pool.acquire()

class BaseRepo:
    def __init__(self):
        self.db = DatabasePool()
    
    async def execute(self, query: str, *args):
        async with await self.db.get_connection() as conn:
            return await conn.execute(query, *args)
    
    async def fetch(self, query: str, *args):
        async with await self.db.get_connection() as conn:
            return await conn.fetch(query, *args)

class UserRepo(BaseRepo):
    async def get_all(self) -> List[User]:
        rows = await self.fetch("SELECT * FROM users ORDER BY username")
        return [User(**row) for row in rows]
    
    async def get_by_id(self, user_id: int) -> Optional[User]:
        rows = await self.fetch("SELECT * FROM users WHERE id = $1", user_id)
        return User(**rows[0]) if rows else None
    
    async def get_by_username(self, username: str) -> Optional[User]:
        rows = await self.fetch("SELECT * FROM users WHERE username = $1", username)
        return User(**rows[0]) if rows else None
    
    async def create(self, request: CreateUserRequest) -> User:
        await self.execute(
            "INSERT INTO users (username, email, password_hash) VALUES ($1, $2, $3)",
            request.username, request.email, request.password_hash
        )
        return await self.get_by_username(request.username)

class PostRepo(BaseRepo):
    async def get_by_user(self, user_id: int) -> List[Post]:
        rows = await self.fetch(
            "SELECT * FROM posts WHERE user_id = $1 ORDER BY created_at DESC", 
            user_id
        )
        return [Post(**row) for row in rows]
    
    async def create(self, post: Post) -> Post:
        await self.execute(
            "INSERT INTO posts (title, content, user_id) VALUES ($1, $2, $3)",
            post.title, post.content, post.user_id
        )
        return post

class CachedUserRepo(UserRepo):
    def __init__(self):
        super().__init__()
        self.cache = {}
    
    async def get_by_id(self, user_id: int) -> Optional[User]:
        if user_id in self.cache:
            return self.cache[user_id]
        
        user = await super().get_by_id(user_id)
        if user:
            self.cache[user_id] = user
        return user
''',
        "full_path": "/project/api/database.py"
    },
    {
        "path": "api/models.py",
        "content": '''
from dataclasses import dataclass
from typing import Optional
from datetime import datetime

@dataclass
class User:
    id: int
    username: str
    email: str
    password_hash: str
    created_at: Optional[datetime] = None
    is_active: bool = True
    
    def display_name(self) -> str:
        return f"@{self.username}"
    
    def is_admin(self) -> bool:
        return self.username in ["admin", "root"]

@dataclass
class Post:
    id: int
    title: str
    content: str
    user_id: int
    created_at: Optional[datetime] = None
    published: bool = True
    
    def excerpt(self, length: int = 150) -> str:
        return self.content[:length] + "..." if len(self.content) > length else self.content
    
    def word_count(self) -> int:
        return len(self.content.split())

@dataclass
class CreateUserRequest:
    username: str
    email: str
    password_hash: str
    
    def validate(self) -> bool:
        return (
            len(self.username) >= 3 and
            "@" in self.email and
            len(self.password_hash) > 0
        )

@dataclass
class APIResponse:
    success: bool
    data: Optional[dict] = None
    message: Optional[str] = None
    
    @classmethod
    def ok(cls, data: dict, message: str = "Success"):
        return cls(success=True, data=data, message=message)
    
    @classmethod
    def error(cls, message: str):
        return cls(success=False, message=message)
''',
        "full_path": "/project/api/models.py"
    }
]

# Create the graph
graph = GraphGenerator(sample_files)
search = GraphSearchTool(graph)

print(f"📊 Created graph: {len(search.nodes_map)} nodes, {len(search.edges_list)} edges")
print(f"📋 Categories: {list(search.get_statistics()['node_categories'].keys())}")

GraphGenerator: Determined project root: /
Identified project type: python
📊 Created graph: 67 nodes, 80 edges
📋 Categories: ['directory', 'module', 'class', 'method', 'external_symbol']


## 🎯 The New API: Every Method Returns a Subgraph

In [3]:
# 🔍 Search for database-related code -> returns GraphGenerator
db_subgraph = search.fuzzy_search("database", depth=2)

print(f"🔍 Database search: {len(db_subgraph.all_nodes_data)} nodes, {len(db_subgraph.all_edges_data)} edges")
print(f"📋 Node types: {[n['category'] for n in db_subgraph.all_nodes_data[:5]]}")

# 🎨 Direct visualization (if ipysigma available)
try:
    db_viz = db_subgraph.visualize(height=400, node_color="category")
    print("✅ Visualization ready!")
    display(db_viz)
except ImportError:
    print("⚠️ Install ipysigma for visualization: pip install ipysigma")

GraphGenerator: Determined project root: /Users/adithyaskolavi/projects/git-repo-mcp/gitvizz/examples
Identified project type: unknown
🔍 Database search: 46 nodes, 54 edges
📋 Node types: ['method', 'method', 'external_symbol', 'method', 'class']
✅ Visualization ready!


Sigma(nx.DiGraph with 46 nodes and 49 edges)

In [4]:
# 🏛️ Filter by category -> returns GraphGenerator  
classes_subgraph = search.filter_by_category(["class"], depth=1)

print(f"🏛️ Classes: {len(classes_subgraph.all_nodes_data)} nodes")
for node in classes_subgraph.all_nodes_data:
    if node['category'] == 'class':
        print(f"   • {node['name']} in {node['file']}")

# Direct visualization
try:
    classes_viz = classes_subgraph.visualize(height=300, node_color="category")
    print("\n✅ Classes visualization:")
    display(classes_viz)
except ImportError:
    print("⚠️ Visualization not available")

GraphGenerator: Determined project root: /Users/adithyaskolavi/projects/git-repo-mcp/gitvizz/examples
Identified project type: unknown
🏛️ Classes: 12 nodes
   • PostRepo in api/database.py
   • Post in api/models.py
   • APIResponse in api/models.py
   • APIServer in api/server.py
   • AuthManager in api/auth.py
   • CachedUserRepo in api/database.py
   • TokenValidator in api/auth.py
   • User in api/models.py
   • CreateUserRequest in api/models.py
   • UserRepo in api/database.py
   • DatabasePool in api/database.py
   • BaseRepo in api/database.py

✅ Classes visualization:


Sigma(nx.DiGraph with 12 nodes and 3 edges)

## 🔗 Relationship Analysis

In [5]:
# Find inheritance relationships -> returns GraphGenerator
inheritance_subgraph = search.find_by_relationship(["inherits"], depth=1)

print(f"🏗️ Inheritance subgraph: {len(inheritance_subgraph.all_nodes_data)} nodes")
if hasattr(inheritance_subgraph, '_search_metadata'):
    centers = inheritance_subgraph._search_metadata.get('center_nodes', [])
    print(f"📋 Inheritance relationships found: {len(centers)} nodes involved")

# Visualize inheritance
try:
    inherit_viz = inheritance_subgraph.visualize(height=300, node_color="category")
    print("\n🎨 Inheritance visualization:")
    display(inherit_viz)
except ImportError:
    print("⚠️ Visualization not available")

GraphGenerator: Determined project root: /Users/adithyaskolavi/projects/git-repo-mcp/gitvizz/examples
Identified project type: unknown
🏗️ Inheritance subgraph: 4 nodes
📋 Inheritance relationships found: 4 nodes involved

🎨 Inheritance visualization:


Sigma(nx.DiGraph with 4 nodes and 3 edges)

In [6]:
# Get neighbors of a specific node -> returns GraphGenerator
api_server_id = "api.server.APIServer"
neighbors_subgraph = search.get_neighbors(api_server_id, depth=2)

print(f"🔗 Neighbors of APIServer: {len(neighbors_subgraph.all_nodes_data)} nodes")
if hasattr(neighbors_subgraph, '_search_metadata'):
    neighbor_count = neighbors_subgraph._search_metadata.get('neighbor_count', 0)
    print(f"   Direct neighbors: {neighbor_count}")

# Visualize neighborhood
try:
    neighbors_viz = neighbors_subgraph.visualize(height=350, node_color="category")
    print("\n🎨 Neighborhood visualization:")
    display(neighbors_viz)
except ImportError:
    print("⚠️ Visualization not available")

GraphGenerator: Determined project root: /Users/adithyaskolavi/projects/git-repo-mcp/gitvizz/examples
Identified project type: unknown
🔗 Neighbors of APIServer: 18 nodes
   Direct neighbors: 4

🎨 Neighborhood visualization:


Sigma(nx.DiGraph with 18 nodes and 17 edges)

## 🧠 LLM Context Generation from Subgraphs

In [7]:
# Generate context from single subgraph
auth_subgraph = search.fuzzy_search("auth", depth=1)
single_context = GraphSearchTool.build_llm_context(auth_subgraph, include_code=True)

print("🤖 Single subgraph LLM context:")
print("=" * 50)
print(single_context[:600] + "..." if len(single_context) > 600 else single_context)
print("=" * 50)

GraphGenerator: Determined project root: /Users/adithyaskolavi/projects/git-repo-mcp/gitvizz/examples
Identified project type: unknown
🤖 Single subgraph LLM context:
# Search Result 1: fuzzy_search
Query: 'auth'
Nodes: 20, Edges: 12

Module {api.server.APIServer.__init__}
File: api/server.py
Defines:

__init__ (method) — lines 8–12
Relationship: api.server.APIServer → api.server.APIServer.__init__ (defines_method)
Relationship: api.server.APIServer.__init__ → fastapi.FastAPI (calls)
Relationship: api.server.APIServer.__init__ → auth.AuthManager (calls)

Code:

```
def __init__(self):
    self.app = FastAPI(title='Blog API')
    self.auth = AuthManager()
    self.user_repo = UserRepo()
    self.post_repo = PostRepo()
```

\
Module {conn.execute}
File: None
...


In [8]:
# Generate context from multiple subgraphs
multi_context = GraphSearchTool.build_llm_context([
    db_subgraph,
    classes_subgraph, 
    auth_subgraph
], include_code=True, max_code_length=200)

print(f"🤖 Multi-subgraph context: {len(multi_context)} characters")
print("\n📋 Context preview (first 800 chars):")
print("=" * 50)
print(multi_context[:800] + "...")
print("=" * 50)

🤖 Multi-subgraph context: 25669 characters

📋 Context preview (first 800 chars):
# Search Result 1: fuzzy_search
Query: 'database'
Nodes: 46, Edges: 54

Module {api.database.BaseRepo.__init__}
File: api/database.py
Defines:

__init__ (method) — lines 18–19
Relationship: api.database.BaseRepo → api.database.BaseRepo.__init__ (defines_method)
Relationship: api.database.BaseRepo.__init__ → api.database.DatabasePool (calls)

Code:

```
def __init__(self):
    self.db = DatabasePool()
```

\
Module {api.database.CachedUserRepo.__init__}
File: api/database.py
Defines:

__init__ (method) — lines 65–67
Relationship: api.database.CachedUserRepo → api.database.CachedUserRepo.__init__ (defines_method)
Relationship: api.database.CachedUserRepo.__init__ → super().__init__ (calls)

Code:

```
def __init__(self):
    super().__init__()
    self.cache = {}
```

\
Module {conn.execute}...


## 🔄 Chaining Operations

In [9]:
# Create new search tool from database subgraph
db_search = GraphSearchTool(db_subgraph)

# Search within the database subgraph
user_in_db = db_search.fuzzy_search("user", depth=1)

print(f"🔍 Original graph: {len(search.nodes_map)} nodes")
print(f"📊 Database subgraph: {len(db_subgraph.all_nodes_data)} nodes")
print(f"🎯 User search in DB subgraph: {len(user_in_db.all_nodes_data)} nodes")

# Visualize the focused search result
try:
    chain_viz = user_in_db.visualize(height=300, node_color="category")
    print("\n🎨 Chained search visualization:")
    display(chain_viz)
except ImportError:
    print("⚠️ Visualization not available")

GraphGenerator: Determined project root: /Users/adithyaskolavi/projects/git-repo-mcp/gitvizz/examples
Identified project type: unknown
🔍 Original graph: 67 nodes
📊 Database subgraph: 46 nodes
🎯 User search in DB subgraph: 20 nodes

🎨 Chained search visualization:


Sigma(nx.DiGraph with 20 nodes and 16 edges)

## 🔥 Advanced Analysis

In [10]:
# Find complexity hotspots -> returns GraphGenerator
hotspots_subgraph = search.get_high_connectivity_nodes(min_connections=3, depth=1)

print(f"🔥 Complexity hotspots: {len(hotspots_subgraph.all_nodes_data)} nodes")
if hasattr(hotspots_subgraph, '_search_metadata'):
    details = hotspots_subgraph._search_metadata.get('connectivity_details', [])
    print("\n📊 Top connected nodes:")
    for detail in details[:3]:
        print(f"   • {detail['node_id']}: {detail['connections']} connections")

# Visualize hotspots
try:
    hotspots_viz = hotspots_subgraph.visualize(height=350, node_color="category")
    print("\n🎨 Complexity hotspots visualization:")
    display(hotspots_viz)
except ImportError:
    print("⚠️ Visualization not available")

GraphGenerator: Determined project root: /Users/adithyaskolavi/projects/git-repo-mcp/gitvizz/examples
Identified project type: unknown
🔥 Complexity hotspots: 18 nodes

📊 Top connected nodes:
   • api.server.APIServer.setup_routes: 8 connections
   • api.auth.AuthManager: 7 connections
   • api.database.BaseRepo: 7 connections

🎨 Complexity hotspots visualization:


Sigma(nx.DiGraph with 18 nodes and 19 edges)

In [11]:
# Find paths between components -> returns GraphGenerator
if len(search.nodes_map) >= 2:
    node_list = list(search.nodes_map.keys())
    source = "api.server.APIServer"
    target = "api.auth.AuthManager"
    
    paths_subgraph = search.find_paths(source, target, max_paths=3)
    
    print(f"🛤️ Paths between {source} and {target}:")
    print(f"   Subgraph: {len(paths_subgraph.all_nodes_data)} nodes")
    
    if hasattr(paths_subgraph, '_search_metadata'):
        paths_found = paths_subgraph._search_metadata.get('paths_found', [])
        print(f"   Found {len(paths_found)} paths:")
        for i, path in enumerate(paths_found, 1):
            print(f"   Path {i}: {' → '.join(path)}")
    
    # Visualize paths
    try:
        paths_viz = paths_subgraph.visualize(height=300, node_color="category")
        print("\n🎨 Paths visualization:")
        display(paths_viz)
    except ImportError:
        print("⚠️ Visualization not available")

GraphGenerator: Determined project root: /Users/adithyaskolavi/projects/git-repo-mcp/gitvizz/examples
Identified project type: unknown
🛤️ Paths between api.server.APIServer and api.auth.AuthManager:
   Subgraph: 0 nodes
   Found 0 paths:

🎨 Paths visualization:


Sigma(nx.DiGraph with 0 nodes and 0 edges)

## 📊 Statistics and Summary

In [12]:
# Get comprehensive statistics
stats = search.get_statistics()

print("📈 Graph Statistics:")
print(f"   Total nodes: {stats['total_nodes']}")
print(f"   Total edges: {stats['total_edges']}")
print(f"   Average connectivity: {stats['average_degree']:.2f}")
print(f"   Connected components: {stats['weakly_connected_components']}")

print("\n📊 Node categories:")
for category, count in stats['node_categories'].items():
    percentage = (count / stats['total_nodes']) * 100
    print(f"   {category}: {count} ({percentage:.1f}%)")

print("\n🔗 Relationship types:")
for rel_type, count in stats['relationship_types'].items():
    print(f"   {rel_type}: {count}")

📈 Graph Statistics:
   Total nodes: 67
   Total edges: 80
   Average connectivity: 2.39
   Connected components: 1

📊 Node categories:
   directory: 1 (1.5%)
   module: 4 (6.0%)
   class: 12 (17.9%)
   method: 19 (28.4%)
   external_symbol: 31 (46.3%)

🔗 Relationship types:
   contains_module: 4
   defines_class: 12
   defines_method: 19
   calls: 42
   inherits: 3


## 🎉 API Benefits Demo

In [13]:
print("✨ NEW API BENEFITS:")
print("="*50)
print()

print("🎯 Every method returns GraphGenerator:")
db_result = search.fuzzy_search("database")
print(f"   search.fuzzy_search('database') → {type(db_result).__name__}")

class_result = search.filter_by_category(['class'])
print(f"   search.filter_by_category(['class']) → {type(class_result).__name__}")

neighbor_result = search.get_neighbors('api.server.APIServer')
print(f"   search.get_neighbors(...) → {type(neighbor_result).__name__}")

print("\n🎨 Direct visualization:")
print("   subgraph.visualize()  # Works immediately!")

print("\n🔄 Easy chaining:")
chained = GraphSearchTool(db_result).fuzzy_search('user')
print(f"   GraphSearchTool(db_result).fuzzy_search('user') → {len(chained.all_nodes_data)} nodes")

print("\n🤖 LLM context generation:")
context = GraphSearchTool.build_llm_context([db_result, class_result])
print(f"   build_llm_context([subgraph1, subgraph2]) → {len(context)} chars")

print("\n✅ No boilerplate code needed!")
print("✅ Consistent API across all methods!")
print("✅ Chainable and composable!")
print("✅ Ready for visualization!")

✨ NEW API BENEFITS:

🎯 Every method returns GraphGenerator:
GraphGenerator: Determined project root: /Users/adithyaskolavi/projects/git-repo-mcp/gitvizz/examples
Identified project type: unknown
   search.fuzzy_search('database') → GraphGenerator
GraphGenerator: Determined project root: /Users/adithyaskolavi/projects/git-repo-mcp/gitvizz/examples
Identified project type: unknown
   search.filter_by_category(['class']) → GraphGenerator
GraphGenerator: Determined project root: /Users/adithyaskolavi/projects/git-repo-mcp/gitvizz/examples
Identified project type: unknown
   search.get_neighbors(...) → GraphGenerator

🎨 Direct visualization:
   subgraph.visualize()  # Works immediately!

🔄 Easy chaining:
GraphGenerator: Determined project root: /Users/adithyaskolavi/projects/git-repo-mcp/gitvizz/examples
Identified project type: unknown
   GraphSearchTool(db_result).fuzzy_search('user') → 14 nodes

🤖 LLM context generation:
   build_llm_context([subgraph1, subgraph2]) → 13516 chars

✅ No bo

## 📝 Summary

### ✨ Key Features:

- **🎯 Consistent API**: Every method returns a `GraphGenerator` subgraph
- **🎨 Direct Visualization**: Call `.visualize()` on any result
- **🔄 Chainable**: Use subgraphs as input to new `GraphSearchTool` instances
- **🤖 LLM Ready**: Generate formatted context from single or multiple subgraphs
- **🔍 Rich Search**: Fuzzy search, category filters, relationship analysis
- **🛤️ Path Analysis**: Find connections between components
- **🔥 Complexity Analysis**: Identify hotspots and high-connectivity nodes

### 🚀 Usage Pattern:

```python
# Search
subgraph = search.fuzzy_search("database")

# Visualize
subgraph.visualize()

# Chain
focused = GraphSearchTool(subgraph).fuzzy_search("user")

# Generate context
context = GraphSearchTool.build_llm_context([subgraph, focused])
```

**Clean, simple, powerful!** 🎉